<a href="https://colab.research.google.com/github/AloofBuddha/neural-nets-zero-to-hero/blob/main/makemore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()

words[:10]

# 32k example
len(words)

# min length 2
min(len(w) for w in words)

# max length 15
max(len(w) for w in words)

# we want to think about the bigrams - what is the frequency of any given letter being followed by another letter

# break down word to bigrams
b = {} # keep track of counts
for w in words:
    # special characters for 'start' / 'end'
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

counts = sorted(b.items(), key=lambda kv: -kv[1])


FileNotFoundError: [Errno 2] No such file or directory: 'names.txt'

In [ ]:
words[:10]

In [ ]:
# bigram count matrix
N = torch.zeros((27, 27), dtype=torch.int32)

# list of chars a-z
chars = sorted(list(set(''.join(words))))

# lookup table char -> index in a-z (including '.' delimiter character for first)
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

# we want a index -> char table as well
itos = {i: s for s,i in stoi.items()}

for w in words:
    # special characters for 'start' / 'end'
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        bigram = (ch1, ch2)
        N[ix1, ix2] += 1

In [ ]:
# visualizer - we have counts for our entire data set that captures all the bigrams, including metacharacters like 'start token' and 'end token'

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off');

Even the most basic analysis can tell us things like:

- The most frequent start letter is 'a'
- the most frequent end letters are 'a' and 'n'
- 'an' is our most frequent bigram
- some of our data points are pointless - a word will never start with the `<E>` token for example, nor end with `<S>` - room to optimize!

N

In [ ]:
# N is our 27 x 27 nigram tensor
N
# our first row represents the counts for [aa, ab, ac, ad ... a.]
N[0]

# cast our sample into floats (was int32) and normalize by total counts
p = N[0].float()
p = p / p.sum()
# p now represents the frequencies of a followed by another character, as compared against all characters, summing in total to 1
p

In [ ]:
# for what we want to do next we need predictable random #s
g = torch.Generator().manual_seed(2147483647) # manual seed is just to make this 'trackable' and not change on each run
p = torch.rand(3, generator=g)
# p holds 3 random #s now between 0 and 1
p = p / p.sum()
# p now equals 3 random normalized values (sum to 1)
print(p)
# now we can use this random distro split in 3 to produce data matching that distro using torch.multinomial
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g)
# by setting the sample size to 1 we can now get a single value from [0, 1, 2] with the generated distribution
ix.item()

In [ ]:
# optimization - turn out predictions into a tensor instead of doing it by hand each time
# N+1 is model smoothing so nll is never -Infinity
P = (N+1).float()

# understanding sum/keepdim
# S = P.sum() # will sum the entire 27 x 27 matrix and return a scalar
# S = P.sum(0) # will sum the matrix by col and return a 27 elem array
# S = P.sum(1) # will sum the matrix by row and return a 27 elem array
# S = P.sum(0, keepdim=True) # will sum the matrix by col and return a 1 x 27 matrix
# S = P.sum(1, keepdim=True) # will sum the matrix by row and return a 27 x 1 matrix

# ultimately this is what we want for matrix multipliction because it is transposed
# remember - broadcasting rules
# 27 x 27
# 27 x 1
# our final matrix S has 27 cols, each with a single row holding the sum of the equivalent row at P
print(P.shape, S.shape)

# each row represents a % frequency for that letter followed by all other letters (normalized)
P /= P.sum(1, keepdim=True)


# frequency per letter map
plt.figure(figsize=(16,16))
plt.imshow(P, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, "{:.2f}".format(P[i, j].item()), ha="center", va="top", color='gray')
plt.axis('off');

In [ ]:
g = torch.Generator().manual_seed(2147483647)

# first for a baseline let's see how our bigram model performs with uniform distribution
# (as if every letter is as likely to follow as any other)

names = []

for i in range(30):
  ix = 0
  name = []

  while True:
    p = torch.ones(27) / 27.0
    # given that frequency distribution, get a random index
    ix = torch.multinomial(p, num_samples=1,
                           replacement=True, generator=g).item()
    name.append(itos[ix])
    # stop when the next random letter chosen is an end char symbol (. = <E>)
    if ix == 0:
      names.append(''.join(name))
      break

# wow, these are some pretty bad names. it has no concept of length
names

In [ ]:
g = torch.Generator().manual_seed(2147483647)

# now let's apply this to our bigram problem
# start at 0 (. = <S>)
names = []

for i in range(30):
  ix = 0
  name = []

  while True:
    p = P[ix]
    # given that frequency distribution, get a random index
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    name.append(itos[ix])
    # stop when the next random letter chosen is an end char symbol (. = <E>)
    if ix == 0:
      names.append(''.join(name))
      break

names


these names are a bit better just because of the bigram frequency approach

names are only so long and often start or end with an a or d as the model predicts

but these don't actually read like names, especially the single letter ones

How can we *train* our model to get better results using NN?

Well first things first we need a way to measure our model's performance - NLL

In [ ]:
# GOAL: maximize likelihood of the data w.r.t. model parameters (statistical modeling)
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log(a) + log(b) + log(c)
log_likelihood = 0.0
n = 0

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    # print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

Now let's cast our bigram model into the language of neural nets

In [ ]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(ch1, ch2)
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

xs, ys

our x -> y are both represented as integer mappings, so ".emma" = [value for . (0), value for 'e' (5) ...]

we have a total of 27 characters

integers don't make so much sense to feed as the value into the input layer of a NN so we will utilize 'one hot encoding'

in one hot encoding we represent the entire datatype as an array of length 27 and 0 for all indices except the one that matches

so e = 5 = [0 0 0 0 1 0 0 0 ...]

In [ ]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
xenc

In [ ]:
plt.imshow(xenc)

xenc.dtype

In [ ]:
W = torch.randn((27, 27))

# @ is pytorch for matrix mult
# (5, 27) @ (27, 27) = (27, 5)
# results in a 5 x 27 array representng input layer "emma" (xs) modified by ws (weights)
# exp is our way of handling negative numbers. Each value now represents the count
# logits - log counts
logits = xenc @ W
# this is now equivalent to our counts matrix N
counts = logits.exp()
# probabilities the given letter follows
probs = counts / counts.sum(1, keepdim=True)
# represents how likely each character is to come next
probs

this is the simplest possible NN. It is an input layer, one hidden layer to turn it into a 27 and an output layer to transform it back to the input size. the input it modified by the weights resulting in the output

Ex:

. -> e
input is . -> 0 -> one hot encoding -> probs of next letter:

[0.0214, 0.0210, 0.0137, 0.0787, 0.0103, 0.0079, 0.0352, 0.0188, 0.0283,
0.2689, 0.0029, 0.0203, 0.0246, 0.0863, 0.0320, 0.0099, 0.0078, 0.0348,
0.0086, 0.0307, 0.0338, 0.0311, 0.0432, 0.0131, 0.0204, 0.0438, 0.0527]

prob . -> . is 2%
     . -> m is 8%

sums to 1

In [ ]:
nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item()  # input character index
  y = ys[i].item()  # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

So we can evaluate the negative log likelihood of each example based on the input weights

so how do we now run this repeatedly on itself so we can minimize nll?

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

W

In [ ]:
# forward pass
# input to the network: one-hot encoding
xenc = F.one_hot(xs, num_classes=27).float()
logits = xenc @ W  # predict log-counts
counts = logits.exp()  # counts, equivalent to N
# probabilities for next character
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(5), ys].log().mean()

loss.item()

In [ ]:
# backward pass
W.grad = None  # set to zero the gradient
loss.backward()

So what did we do?

example: . -> e

fed input . -> 0 -> one hot encode -> applied weights (randomized initially according to normal dist) -> resulting prob matrix guessing the next letter -> compared to actual next letter 'e' we get a loss function representing how well we did -> run a backwards pass of our NN to readjust the weights to get a slightly better loss function the next time the same example is run on this NN

In [ ]:
# now let's try it on the full dataset

# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for k in range(200):

  # forward pass
  # input to the network: one-hot encoding
  xenc = F.one_hot(xs, num_classes=27).float()
  logits = xenc @ W  # predict log-counts
  counts = logits.exp()  # counts, equivalent to N
  # probabilities for next character
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  # backward pass
  W.grad = None  # set to zero the gradient
  loss.backward()

  # update (-50 represents our learning rate)
  W.data += -50 * W.grad

In [ ]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix = 0
  while True:

    # ----------
    # BEFORE:
    # p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W  # predict log-counts
    counts = logits.exp()  # counts, equivalent to N
    # probabilities for next character
    p = counts / counts.sum(1, keepdims=True)
    # ----------

    ix = torch.multinomial(
        p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

# after one pass it's not significantly better

ok so what did we learn here?

originally we built a model from scratch using our insight of the bigram counts

then we did the same thing but starting from a random set of weights and using gradient descent

after 100 iterations of gradient descent we get the same results as the handcrafted strategy

it effectively learns the bigram counts model